In [4]:
import requests
import json
from os import environ
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
TWITTER_API_KEY = environ.get("TWITTER_API_KEY")
TWITTER_SECRET_KEY = environ.get("TWITTER_SECRET_KEY")
TWITTER_AUTH_BEARER_TOKEN = environ.get("TWITTER_AUTH_BEARER_TOKEN")
TWITTER_ACCESS_TOKEN = environ.get("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_SECRET = environ.get("TWITTER_ACCESS_SECRET")

In [5]:
def create_headers():
    headers = {"Authorization": "Bearer {}".format(TWITTER_AUTH_BEARER_TOKEN)}
    return headers

In [6]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [7]:
def get_tweet_info_url(ids):
    tweet_fields = "tweet.fields=public_metrics"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    separated_ids = "ids=" + ",".join([str(id) for id in ids])
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = f"https://api.twitter.com/2/tweets?{separated_ids}&{tweet_fields}"
    return url

In [8]:
def get_retweeters_url(id):
    return f"https://api.twitter.com/1.1/statuses/retweeters/ids.json?id={id}"


In [17]:
def get_retweets_url(id):
    return f"https://api.twitter.com/1.1/statuses/retweets/{id}.json"

In [66]:
ids = [1376030179848491009]
retweeter_objects = []
retweet_objects = []
tweet_info_objects = []
for id in ids:
    retweeter_objects.extend(connect_to_endpoint(get_retweeters_url(id), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"})["ids"])
    retweet_objects.extend(connect_to_endpoint(get_retweets_url(id), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"}))
tweet_info_objects = connect_to_endpoint(get_tweet_info_url(ids), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"})["data"]

In [36]:
print(retweet_objects)

[{'created_at': 'Sun Mar 28 17:34:24 +0000 2021', 'id': 1376226176687988741, 'id_str': '1376226176687988741', 'text': 'RT @NBA: 🚨 Harrison Barnes catches the full court pass and drills the #TissotBuzzerBeater to win it for the @SacramentoKings! #ThisIsYourTi…', 'truncated': False, 'entities': {'hashtags': [{'text': 'TissotBuzzerBeater', 'indices': [70, 89]}], 'symbols': [], 'user_mentions': [{'screen_name': 'NBA', 'name': 'NBA', 'id': 19923144, 'id_str': '19923144', 'indices': [3, 7]}, {'screen_name': 'SacramentoKings', 'name': 'Sacramento Kings', 'id': 667563, 'id_str': '667563', 'indices': [108, 124]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 20956062, 'id_str': '20956062', 'name': 'Jackie Christie', 'screen_name': 'JackieChristie', 'location'

In [37]:
print(retweet_objects)

[{'created_at': 'Sun Mar 28 17:34:24 +0000 2021', 'id': 1376226176687988741, 'id_str': '1376226176687988741', 'text': 'RT @NBA: 🚨 Harrison Barnes catches the full court pass and drills the #TissotBuzzerBeater to win it for the @SacramentoKings! #ThisIsYourTi…', 'truncated': False, 'entities': {'hashtags': [{'text': 'TissotBuzzerBeater', 'indices': [70, 89]}], 'symbols': [], 'user_mentions': [{'screen_name': 'NBA', 'name': 'NBA', 'id': 19923144, 'id_str': '19923144', 'indices': [3, 7]}, {'screen_name': 'SacramentoKings', 'name': 'Sacramento Kings', 'id': 667563, 'id_str': '667563', 'indices': [108, 124]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 20956062, 'id_str': '20956062', 'name': 'Jackie Christie', 'screen_name': 'JackieChristie', 'location'

In [67]:
print(tweet_info_objects)

[{'id': '1376030179848491009', 'public_metrics': {'retweet_count': 387, 'reply_count': 65, 'like_count': 2341, 'quote_count': 138}, 'text': '🚨 Harrison Barnes catches the full court pass and drills the #TissotBuzzerBeater to win it for the @SacramentoKings! #ThisIsYourTime 🚨 https://t.co/3Yc0YkOc9Z'}]


In [38]:
for object in retweet_objects:
    print(object['id'])

1376226176687988741
1376225868591333379
1376225809933893633
1376224295488159746
1376219007326556160
1376216965879070726
1376216398033207297
1376214237635284997
1376213468215439360
1376209738526978052
1376204288209260545
1376203912261333004
1376201974929055745
1376201484610768897
1376198282179907584
1376198240660557830
1376196764986925058
1376196093260746760


In [59]:
def find_user(id):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    # usernames = "usernames=TwitterDev,TwitterAPI"
    user_fields = "user.fields=description,created_at,public_metrics"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/{}?&{}".format(id, user_fields)
    return url

In [60]:
for id in retweeter_objects:
    user = connect_to_endpoint(find_user(id), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"})["data"]
    print(user)

{'username': 'NadiaaByers', 'created_at': '2010-01-10T02:59:08.000Z', 'public_metrics': {'followers_count': 1437, 'following_count': 501, 'tweet_count': 48729, 'listed_count': 3}, 'description': 'a leader needs followers #uncgalum', 'name': 'Nadia Byers™', 'id': '103459588'}
{'description': '', 'public_metrics': {'followers_count': 622, 'following_count': 2310, 'tweet_count': 310, 'listed_count': 0}, 'name': 'Ato Gregory', 'id': '1247632289426812931', 'username': 'AtoGregory1', 'created_at': '2020-04-07T21:08:28.000Z'}
{'description': 'Aprende a apreciar lo que tienes, antes de que el tiempo te haga apreciar lo que tenías', 'public_metrics': {'followers_count': 266, 'following_count': 1298, 'tweet_count': 25142, 'listed_count': 0}, 'name': 'GARI (PAPALECO)', 'id': '1163867678399959047', 'username': 'GariNarcise', 'created_at': '2019-08-20T17:38:12.000Z'}
{'id': '344007601', 'public_metrics': {'followers_count': 2086, 'following_count': 2671, 'tweet_count': 66084, 'listed_count': 5}, 'n

{'id': '3801586273', 'username': 'shoyoshidasho', 'created_at': '2015-10-06T09:00:55.000Z', 'description': 'インターネットベンチャー界隈/バスケ🏀/beer🍺', 'name': 'Sho Yoshida', 'public_metrics': {'followers_count': 148, 'following_count': 1058, 'tweet_count': 4373, 'listed_count': 1}}
{'description': '🍟 creative: @WKNYC / 🏀 podcasts: @TheAthletic / 🎙co-founder: @CountTheDings •tweets are my own• travonneledwards@gmail.com', 'public_metrics': {'followers_count': 20826, 'following_count': 1064, 'tweet_count': 44997, 'listed_count': 544}, 'id': '3466841966', 'created_at': '2015-09-06T04:44:32.000Z', 'username': 'Travonne', 'name': 'travonne edwards (blk tray)'}
{'description': 'Half naked and almost Famous. Top🐕\u200d🦺 💤VILLE', 'public_metrics': {'followers_count': 4030, 'following_count': 5004, 'tweet_count': 512875, 'listed_count': 30}, 'name': 'old sport ⁶𓅓𓆲', 'id': '125053452', 'username': '_isheikh', 'created_at': '2010-03-21T14:36:02.000Z'}
{'id': '253867095', 'public_metrics': {'followers_count': 95

{'username': 'diegopereyracl1', 'description': '', 'public_metrics': {'followers_count': 0, 'following_count': 7, 'tweet_count': 3, 'listed_count': 0}, 'name': 'diegopereyraclase@gmail.com', 'id': '1343569460641345536', 'created_at': '2020-12-28T14:48:36.000Z'}
{'public_metrics': {'followers_count': 74, 'following_count': 115, 'tweet_count': 2451, 'listed_count': 6}, 'description': 'I love basketball&UtahJazz!  応援歴25年NBA博士！', 'id': '272270218', 'username': 'yujifivesix', 'name': 'すとっく@memo13', 'created_at': '2011-03-26T05:05:21.000Z'}
{'description': '21, Barnsley, Insta- bradsimms40', 'public_metrics': {'followers_count': 331, 'following_count': 428, 'tweet_count': 61336, 'listed_count': 0}, 'id': '1008422513116893184', 'created_at': '2018-06-17T18:53:57.000Z', 'username': 'BradFCB10S', 'name': 'Brad'}
{'created_at': '2010-04-15T09:40:24.000Z', 'id': '133225702', 'description': 'I work for🇯🇵HealthcareFoodservice Co.,Ltd./Single/Fullofcuriosity🧗🏻\u200d♀️/Solotraveller✈️ /Luv💘Art /Balle

{'description': 'CEO & Founder of KajoItech & Chochote magazine.. Basketballer, programmer website & mobile developer, graphics designer.. Always like doin what makes me happy..', 'id': '256243268', 'name': 'Kayumba Legrand', 'username': 'KayumbaLeGrand', 'public_metrics': {'followers_count': 490, 'following_count': 403, 'tweet_count': 3589, 'listed_count': 4}, 'created_at': '2011-02-22T23:29:50.000Z'}
{'public_metrics': {'followers_count': 234, 'following_count': 1378, 'tweet_count': 3003, 'listed_count': 1}, 'description': 'サブ垢です。バスケ、アニメ、えろいことをRTする。本垢しりたい人なんていないと思うけど知りたいなら勝手に調べてください', 'id': '4708796761', 'username': 'krkr0519', 'name': '🤪', 'created_at': '2016-01-04T13:05:42.000Z'}
{'description': '', 'name': 'TeamCarterSauce', 'id': '1360794935859978240', 'public_metrics': {'followers_count': 3, 'following_count': 23, 'tweet_count': 165, 'listed_count': 0}, 'username': 'TeamCarterSauc1', 'created_at': '2021-02-14T03:36:17.000Z'}
{'public_metrics': {'followers_count': 485, 'following

In [61]:
ids = [1376196093260746760]
retweeter_objects = []
retweet_objects = []
tweet_info_objects = []
for id in ids:
    retweeter_objects.extend(connect_to_endpoint(get_retweeters_url(id), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"})["ids"])
    retweet_objects.extend(connect_to_endpoint(get_retweets_url(id), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"}))
tweet_info_objects = connect_to_endpoint(get_tweet_info_url(ids), {"Authorization": f"Bearer {TWITTER_AUTH_BEARER_TOKEN}"})["data"]

In [62]:
print(tweet_info_objects)

[{'id': '1376196093260746760', 'public_metrics': {'retweet_count': 387, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': 'RT @NBA: 🚨 Harrison Barnes catches the full court pass and drills the #TissotBuzzerBeater to win it for the @SacramentoKings! #ThisIsYourTi…'}]


In [64]:
print(retweet_objects)

[]
